# TP2 (NLP)

In [131]:
#https://github.com/edupoux/MVA_2019_SL/tree/master/TD_%232

#https://cs.brown.edu/courses/csci1460/assets/files/parsing.pdf

import pandas as pd
import numpy as np

import pickle
from operator import itemgetter
import re

path_to_data = "data/"

In [132]:
file_corpus = open(path_to_data+"SEQUOIA_treebank","r")
corpus = []
for line in file_corpus:
    corpus.append(line)
file_corpus.close()

In [133]:
print(corpus[0:10])

['( (SENT (NP (NPP Gutenberg))))\n', '( (SENT (NP-SUJ (DET Cette) (NC exposition)) (VN (CLO-A_OBJ nous) (V apprend)) (Ssub-OBJ (CS que) (PP-MOD (P dès) (NP (DET le) (ADJ XIIe) (NC siècle))) (PONCT ,) (PP-MOD (P à) (NP (NPP Dammarie-sur-Saulx))) (PONCT ,) (PP-MOD (P entre) (NP (ADJ autres) (NC sites))) (PONCT ,) (NP-SUJ (DET une) (NC industrie) (AP (ADJ métallurgique))) (VN (V existait))) (PONCT .)))\n', "( (SENT (PP-MOD (ADV à_peu_près) (P+D au) (NP (ADJ même) (NC moment) (Ssub (CS que) (NP-SUJ (NPP Gutenberg)) (VN (V inventait)) (NP-OBJ (DET l') (NC imprimerie))))) (PONCT ,) (NP-SUJ (NPP Gillet) (NPP Bonnemire)) (VN (V créait)) (PP-MOD (P en) (NP (NC 1450))) (NP-OBJ (DET la) (ADJ première) (NC forge)) (PP-MOD (P à) (NP (NPP Saint-Dizier))) (PONCT ,) (PP-MOD (P à) (NP (DET l') (ADJ actuel) (NC emplacement) (PP (P+D du) (NP (NPP CHS))))) (PONCT .)))\n", "( (SENT (ADV Ensuite) (PONCT ,) (VN (V fut) (VPP installée)) (NP-SUJ (DET une) (ADJ autre) (NC forge)) (PP-P_OBJ (P à) (NP (DET la) (N

# Splitting the corpus into train/dev/test

In [134]:
frac_train = 0.8
frac_dev = 0.1
frac_test = 1-frac_train-frac_dev

N = len(corpus)
nb_train = int(N*frac_train)
nb_dev = int(N*frac_dev)

corpus_train = corpus[:nb_train]
corpus_dev = corpus[nb_train:nb_train+nb_dev]
corpus_test = corpus[nb_train+nb_dev:]

In [135]:
print(N,nb_train,nb_dev)

3100 2480 310


# Extracting a PCGF from the training corpus

In [136]:
def add(dico, word, tag): 
    #incrementing dico[word][tag], word is a string, tag is a string or a list (will be converted to a tuple in such case)
    
    if type(tag)==list:
        tag = tuple(tag)
    if word in dico.keys():
        if tag in dico[word].keys():
            dico[word][tag]+=1
        else:
            dico[word][tag] = 1
    else:
        dico[word] = {tag:1}
        
def non_functional_tag(functional_tag):
    tag = ""
    for caract in functional_tag:
        if caract=="-":
            break
        tag+=caract
    return tag

def normalize_counts(dico):
    #convert counts to probabilities
    #ie: perform for each idx, the transformation below
    #dico[idx] = {i:c,j:d} ->  dico[idx] = {i:c/(c+d),j:d/(c+d)}
    for (word,tags_counts) in dico.items():
        total_counts = np.sum(list(tags_counts.values()))
        for tag in tags_counts.keys():
            dico[word][tag] /= total_counts

In [137]:
def PGFG(corpus, return_counts_tokens=False):
    
    #corpus is postagged
    
    grammar = {}
    lexicon = {}
    
    for postagged_sent in corpus:
        #print(postagged_sent)
        
        sent = postagged_sent.split() #into a list
        hierarchy = [] #index = number of opened brackets since the beginning of the sentence
                       #hierarchy[index] = list of tags pointed by root tag hierarchy[index-1]
        hierarchy.append([]) #list for level 0
        
        level = 0 #current difference between the number of opened brackets (minus the first one) and the number of closed brackets
        current_tag = None
        
        for bloc in sent:
            
            if (bloc[0]=="("): #then the bloc is introducing a new postag
                
                postag = non_functional_tag(bloc[1:])  #we add it to the hierarchy
                if level<len(hierarchy): #there is already one postag as its level
                    hierarchy[level].append(postag)
                else: #first postag as its level
                    hierarchy.append([postag])
                #print(hierarchy)
                level += 1 #since we opened a new bracket
                current_tag = postag #saved in order to add the word to the lexicon
                
            else: #then the bloc is introducing the word name and the number of closing brackets
                
                word = ""
                nb_closing_brackets = 0
                for caract in bloc:
                    if (caract==")"):
                        nb_closing_brackets += 1
                    else:
                        word += caract
                add(lexicon, word, current_tag) #adding the pair (word,postag) to the lexicon
                level -= nb_closing_brackets #since we closed a bracket
                
                for k in range(nb_closing_brackets-1,0,-1): #at least 2 brackets closed -> new grammar rule defined
                    root = hierarchy[-2][-1] #root tag
                    if root=='': #if the root is the beginning of the sentence
                        break
                    tags = hierarchy[-1] #child tags
                    add(grammar, root, tags) #adding the rule to the grammar
                    hierarchy.pop() #popping from the hierarchy the childs list
                    
                    #print(root,tags)
                    #print(hierarchy)
           
    #building a dictionnary computing counts of tokens (disregarding tags)
    if return_counts_tokens:
        counts_tokens = {word:np.sum(list(tags_counts.values())) for (word,tags_counts) in lexicon.items()}
        
    #convert counts into probabilities of grammar rules (from a given root) / tags (for a given word)
    normalize_counts(grammar)
    normalize_counts(lexicon)
    
    if return_counts_tokens:
        return grammar, lexicon, counts_tokens
    else:
        return grammar, lexicon

In [138]:
grammar, lexicon, counts_tokens = PGFG(corpus_train,return_counts_tokens=True)

In [139]:
#print(counts_tokens)
print(np.sum(list(grammar['COORD'].values())))
print(grammar)
print(lexicon)

1.0
{'NP': {('NC', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD'): 7.034327518289252e-05, ('DET', 'ADJ', 'NC', 'NC', 'PP'): 7.034327518289252e-05, ('DET', 'NC', 'AP', 'PONCT', 'VPpart'): 0.0004220596510973551, ('DET', 'NPP', 'PONCT', 'NP'): 0.00014068655036578504, ('NC', 'PP', 'COORD'): 0.0002813731007315701, ('DET', 'PONCT', 'NC', 'NPP', 'PONCT', 'Srel', 'PONCT'): 7.034327518289252e-05, ('DET', 'ADJ', 'NC', 'PP'): 0.005557118739448509, ('ADV', 'P', 'NC', 'COORD'): 7.034327518289252e-05, ('NC', 'AP', 'PONCT', 'NP', 'PONCT'): 0.0007737760270118177, ('ADJ', 'NC', 'AP', 'PONCT', 'PP'): 7.034327518289252e-05, ('NC', 'ADJ'): 0.0014772087788407428, ('NPP', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD', 'COORD'): 7.034327518289252e-05, ('NC', 'AP', 'AP', 'PP', 'PONCT', 'NP', 'PONCT'): 0.00021102982554867754, ('DET', 'PONCT', 'NC', 'PONCT', 'PP'): 0.00014068655036578504, ('DET', 'NPP', 'VPpart'): 0.0002813731

# Handling OOV (out of vocabulary words)

## a. To find the closest neighbor in the postags corpus of a word having an embedding

In [140]:
#functions imported from https://nbviewer.jupyter.org/gist/aboSamoor/6046170

# Noramlize digits by replacing them with #
DIGITS = re.compile("[0-9]", re.UNICODE)

def case_normalizer(word, dictionary):
    """ In case the word is not available in the vocabulary,
     we can try multiple case normalizing procedure.
     We consider the best substitute to be the one with the lowest index,
     which is equivalent to the most frequent alternative."""
    w = word
    lower = (dictionary.get(w.lower(), 1e12), w.lower())
    upper = (dictionary.get(w.upper(), 1e12), w.upper())
    title = (dictionary.get(w.title(), 1e12), w.title())
    results = [lower, upper, title]
    results.sort()
    index, w = results[0]
    if index != 1e12:
        return w
    return word

def normalize(word, word_id):
    """ Find the closest alternative in case the word is OOV."""
    if not word in word_id:
        word = DIGITS.sub("#", word)
    if not word in word_id:
        word = case_normalizer(word, word_id)

    if not word in word_id:
        return None
    return word

def l2_nearest(embeddings, query_embedding, k):
    """Sorts words according to their Euclidean distance.
       To use cosine distance, embeddings has to be normalized so that their l2 norm is 1.
       indeed (a-b)^2"= a^2 + b^2 - 2a^b = 2*(1-cos(a,b)) of a and b are norm 1"""
    distances = (((embeddings - query_embedding) ** 2).sum(axis=1) ** 0.5)
    sorted_distances = sorted(enumerate(distances), key=itemgetter(1))
    return zip(*sorted_distances[:k])

In [141]:
def build_embeddings_lexicon(words_lexicon, words_with_embeddings_id, embeddings):
    # function returning the embeddings matrix of the words of lexicon having one
    # and the mapping id-word / word-id for these lexicon words

    words_lexicon_in_corpus = [] #words of lexicon having an embedding

    # Embeddings of words of lexicon present in embeddings corpus
    embeddings_lexicon = None
    for word in words_lexicon:
        word = normalize(word, words_with_embeddings_id)
        if not(word is None):
            words_lexicon_in_corpus.append(word)
            word_index = words_with_embeddings_id[word]
            id_word = words_with_embeddings_id[word]
            if embeddings_lexicon is None:
                embeddings_lexicon = embeddings[id_word]
            else:
                embeddings_lexicon = np.vstack([embeddings_lexicon,embeddings[id_word]])

    # Map lexicon words present in embedding corpus to new ids and vice versa
    word_lexicon_id = {w:i for (i, w) in enumerate(words_lexicon_in_corpus)}
    id_word_lexicon = words_lexicon_in_corpus
    
    return embeddings_lexicon, word_lexicon_id, id_word_lexicon

In [142]:
words_with_embeddings, embeddings = pickle.load(open(path_to_data+'polyglot-fr.pkl', "rb"), encoding='bytes') #or "bytes" or latin1"
words_with_embeddings_id = {w:i for (i, w) in enumerate(words_with_embeddings)}   # Map words to indices
words_lexicon = list(lexicon.keys())

embeddings_lexicon, word_lexicon_id, id_word_lexicon = build_embeddings_lexicon(words_lexicon, words_with_embeddings_id, embeddings)
embeddings_lexicon /= np.linalg.norm(embeddings_lexicon,axis=1)[:,None]

print(len(words_lexicon)," words in lexicon")
print(len(id_word_lexicon), " words in lexicon having an embedding")
print(embeddings_lexicon.shape)
print(id_word_lexicon[0:2])

8958  words in lexicon
7427  words in lexicon having an embedding
(7427, 64)
['Grande', 'fatigue']


In [143]:
def closest_word_in_corpus(query):
    #return nearest_neighbor(query, embeddings, word_id, id_word, 3)    
    query = normalize(query,words_with_embeddings_id)
    if not query:
        print("OOV word")
        return None
    query_index = words_with_embeddings_id[query]
    query_embedding = embeddings[query_index]
    indices, distances = l2_nearest(embeddings_lexicon, query_embedding, 1)
    neighbors = [id_word_lexicon[idx] for idx in indices]
    return neighbors[0]

res = closest_word_in_corpus("je")
print(res, res in id_word_lexicon)

je True


## b. To find the closest neighbor in the vocabulary (postags corpus + embeddings corpus) of a word, considering spelling errors

In [144]:
vocabulary = list(words_with_embeddings) + words_lexicon
word_vocab_to_id = {w:i for (i, w) in enumerate(vocabulary)}

In [145]:
def levenstein_damerau_distance(word,word2):
    
    size_word2 = len(word2)
    dist = np.zeros((3,size_word2+1))
    #dist[0,j] = distance from word[:t-1] to word2[:j-1]
    #dist[1,j] = distance from word[:t] to word2[:j-1]
    #dist[2,j] = distance from word[:t+1] to word2[:j-1]
    #where w[:-1] is the void string
    #and where t worths 0 at the beginning and progressively increases up to size_word1-1
    #(enables to reach a linear space complexity, 
    #I do not save the whole matrix of distances between prefixes but only the last two lines)
    
    dist[0,:] = np.arange(size_word2+1) #distance from void string to word2[:j]
    dist[1,0] = 1
    for j in range(1,size_word2+1):
        diff_last_letters =  word[0]!=word2[j-1] #different last letters of prefixes
        dist[1,j] = min([dist[0][j]+1,dist[1][j-1]+1,dist[0][j-1]+diff_last_letters]) 
    
    for i in range(2,len(word)+1):
        
        dist[2][0] = i #distance from word[:i] to void string
        for j in range(1,size_word2+1):
            diff_last_letters =  word[i-1]!=word2[j-1] 
            dist[2,j] = min([dist[1][j]+1,dist[2][j-1]+1,dist[1][j-1]+diff_last_letters]) 
            if j>1: #consider swap too ! 
                if (word[i-1]==word2[j-2])and(word[i-2]==word2[j-1]):
                    dist[2,j] = min(dist[2,j],dist[0,j-2]+1)
        
        dist[0,:] = dist[1,:]
        dist[1,:] = dist[2,:]
        
    return dist[2][size_word2]
        
print(levenstein_damerau_distance("tavupe","tvapu"))

def corrected_word(query):
    
    #TODO : normalize query, and also treebank lexicon words !!!
    #query = DIGITS.sub("#", query)
    #query = case_normalizer(query, word_id)
    
    #le mot a une longueur différente : éliminer les cas ...
    
    candidates = {1:[],2:[],3:[]} #words at distances 1,2,3 from real words
    min_dist = 3 #distance with closest word
    
    for word in words_lexicon: 
        #we look for corrections in treebank corpus at most distance min_dist from query
        
        dist = levenstein_damerau_distance(query,word)
        if dist<=min_dist:
            candidates[dist].append(word)
            min_dist = dist
    
    if len(candidates[1])>0: 
        #there is at least one word in treebank corpus at distance 1, 
        #we return the most frequent of these
        
        idx_most_frequent = np.argmax([counts_tokens[word] for word in candidates[1]])
        return candidates[1][idx_most_frequent]
    
    #####################################
    #if we reached this line
    #all words in treebank corpus are at distance more than 2
    #we look for corrections in embeddings corpus at most distance min_dist from query

    for word in words_with_embeddings:
        dist = levenstein_damerau_distance(query,word)
        if dist<=min_dist:
            candidates[dist].append(word)
            min_dist = dist
        if min_dist==1: 
            #since no word at distance 1 was found previously in treebank corpus,
            #we return the word which has an embedding and accomplished distance 1 with query
            return candidates[1][0]

    #####################################
    #if we reached this line,
    #we found words in treebank/embeddings at at least distance 2 from the query
    
    list_candidates = candidates[min_dist]
    candidates_in_lexicon = []

    for word in list_candidates:
        if word in words_lexicon: candidates_in_lexicon.append(word)

    if len(candidates_in_lexicon)==0: 
        #the min distance is accomplished only by words which have an embedding, we return one of these
        return list_candidates[0]

    #####################################
    #if we reached this line
    #the min distace is accomplished by a word in treebank corpus, we return the most frequent of these
    idx_most_frequent = np.argmax([counts_tokens[word] for word in candidates_in_lexicon])
    return candidates_in_lexicon[idx_most_frequent]            

3.0


In [146]:
def all_tags(lexicon):
    res =  []
    for (word,tags) in lexicon.items():
        res += list(tags.keys())
    return np.unique(res)

set_tags = all_tags(lexicon)
nb_tags = len(set_tags)
    
def tag_oov(oov_word, viz_closest = False):

    if oov_word in words_with_embeddings:
        #look for words of corpus whose embedding is closest to the 
        #embedding of oov_word
        closest_corpus_word = closest_word_in_corpus(oov_word) #, embeddings, words_lexicon
        if viz_closest: print(closest_corpus_word, " is the closest word (meaning) found among lexicon words having an embedding")
        return lexicon[closest_corpus_word]
    
    else: #look for spelling errors
        correction = corrected_word(oov_word)
        
        if correction is None:
            print("no corrected word (spelling) found at damerau-levenshtein distance less than 3")
            return {tag:1/nb_tags for tag in set_tags}
        
        else:  
            print(correction, " is the closest word (spelling) found among words in the lexicon or having an embedding")

            if correction in words_lexicon: #if corrected word in corpus
                if viz_closest: print(correction, " is a word in the lexicon")
                return lexicon[correction]

            else: #if corrected word in embedding corpus
                closest_corpus_word = closest_word_in_corpus(correction) #, embeddings, words_lexicon
                if viz_closest: print(closest_corpus_word, " is the closest word (meaning) found among lexicon words having an embedding")
                return lexicon[closest_corpus_word]

In [147]:
def tag(word, viz_closest = False):
    if word in lexicon:
        return lexicon[word]
    else:
        if viz_closest: print(word," is an OOV")
        return tag_oov(word, viz_closest = viz_closest)
    
#http://www.aclweb.org/anthology/J98-4004

In [148]:
def sentence(postag):
    postag_splitted = postag.split() #into a list
    sent = []
    for bloc in postag_splitted:
        if (bloc[0]=="("):
            continue
        else:
            word = ""
            for caract in bloc:
                if (caract==")"):
                    break
                word += caract
            sent.append(word)
    return ' '.join(sent)

sentences_test = [sentence(postag) for postag in corpus_test]
for word in sentences_test[0].split():
    print(word, tag(word,viz_closest = True))
    print("")

- {'CC': 0.010810810810810811, 'PONCT': 0.9783783783783784, 'ADV': 0.010810810810810811}

Février  is an OOV
Janvier  is the closest word (meaning) found among lexicon words having an embedding
Février {'NC': 1.0}

2005 {'NC': 1.0}

: {'PONCT': 1.0}

le {'CLO': 0.040993788819875775, 'DET': 0.9590062111801242}

parquet {'NC': 1.0}

de {'P': 0.9788445890968267, 'DET': 0.02115541090317331}

Paris {'NPP': 1.0}

requiert  is an OOV
nécessite  is the closest word (meaning) found among lexicon words having an embedding
requiert {'V': 1.0}

un {'DET': 0.9379014989293362, 'PRO': 0.06209850107066381}

non-  is an OOV
non  is the closest word (spelling) found among words in the lexicon or having an embedding
non  is a word in the lexicon
non- {'NC': 0.027777777777777776, 'ADV': 0.9722222222222222}

lieu {'NC': 1.0}

en_faveur_de {'P': 1.0}

Jean {'NPP': 1.0}

Tiberi {'NPP': 1.0}

, {'PONCT': 1.0}

accordé {'VPP': 1.0}

par {'P': 1.0}

le {'CLO': 0.040993788819875775, 'DET': 0.9590062111801242}

j

# CYK Algorithm

In [149]:
#print(sentences_test[:10])